In [122]:
import pandas as pd
import numpy as np
import statistics

In [123]:
import requests
import json

In [124]:
def callApi(method, stock,isEarnings = False ):
    key = "token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    query = "?"
    add = "&"
    last = "/10"
    period = "period=annual"
    if isEarnings:
        url = host + ticker + api_method + last + query + period + add + key 
    else:
        url = host + ticker + api_method + query + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [177]:
def peRatio(quote):
    if "peRatio" in quote.keys() and quote["peRatio"] is not None:
        return float(quote["peRatio"])
    else:
        0.0

In [126]:
def priceToBook(stats):
    if "priceToBook" in stats.keys() and stats["priceToBook"] is not None:
        return float(stats["priceToBook"])
    else:
        return 0.0

In [127]:
def debtToEquity(stats):
    if "debtToEquity" in stats.keys() and stats["debtToEquity"] is not None:
        return float(stats["debtToEquity"])
    else:
        return 0.0
    

In [128]:
def pegRatio(stats):
    if not("pegRatio" in stats.keys()):
        return 0.0
    if stats["pegRatio"] is not None:
        return float(stats["pegRatio"])
    else:
        return 0.0

In [129]:
def sharesoutstanding(stats):
    if not("sharesOutstanding" in stats.keys()):
        return 0.0
    if stats["sharesOutstanding"] is not None:
        return float(stats["sharesOutstanding"])
    else:
        return 0.0

In [130]:
# Current Ratio= Current Assets / Current Liabilities
def currentRatio(balance_sheet):
    if not("balancesheet" in balance_sheet.keys()) or len(balance_sheet["balancesheet"]) == 0:
        return 0.0
    latest_balance_sheet = balance_sheet["balancesheet"][0]
    if not("totalAssets" in latest_balance_sheet.keys()) and not("totalLiabilities" in latest_balance_sheet.keys()):
        return 0.0
    return float(latest_balance_sheet["totalAssets"]/ latest_balance_sheet["totalLiabilities"])
    

In [131]:
def checkErningProgression(earnings):
    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return False
    earningsSet = earnings["earnings"][:-1]
    yearAgoChangePercent = np.array([])
    for earning in earningsSet:
        if not("yearAgoChangePercent" in earning.keys()):
            return False
        yearAgoChangePercent = np.append(yearAgoChangePercent, earning["yearAgoChangePercent"])
    isDecending = (yearAgoChangePercent == np.sort(yearAgoChangePercent)[::-1]).all()
    return isDecending

In [132]:
def shareholdersEquity(financials):
    if not("financials" in financials.keys()) or len(financials["financials"]) == 0:
        return 0.0
    financial = financials["financials"][0]
    if not("shareholderEquity" in financial.keys()):
        return 0.0
    return financial["shareholderEquity"]

In [133]:
def totalRevenue(financials):
    if not("financials" in financials.keys()) or len(financials["financials"]) == 0:
        return 0.0
    financial = financials["financials"][0]
    if not("totalRevenue" in financial.keys()):
        return 0.0
    return financial["totalRevenue"]

In [134]:
def avgEPS(earnings):
    # Average Ernings 
    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return 0.0
    earningsSet = earnings["earnings"]
    count = 0
    epsSum = 0
    for earning in earningsSet:
        if "actualEPS" in earning.keys():
            epsSum += earning["actualEPS"] 
            count += 1
    return float(epsSum/count)

In [135]:
# COMPOUND ANNUAL GROWTH RATES (CAGR)
def EPS_CAGR(earnings):
    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return 0.0
    earningsSet = earnings["earnings"]
    eps = np.array([])
    for earning in earningsSet: 
        actualEPS = float(earning["actualEPS"])
        if actualEPS <= 0:
            eps = np.append(eps, 0.1) 
        else: 
            eps = np.append(eps, actualEPS)
    periods = len(eps)
    if periods > 1 :
        first = eps[-1]
        last = eps[0] 
        cage = ((last/first)**(1/periods))-1
        return cage
    else:
        return 0

In [136]:
def returnOnEquitySerice(financials):
    #ROE = Net Income / Shareholder Equity
    if not("financials" in financials.keys()) or len(financials["financials"]) == 0:
        return []
    financialsData = financials["financials"]
    returnOnEquity = np.array([])
    for financial in financialsData:
        if "shareholderEquity" in financial.keys() and "netIncome" in financial.keys() : 
            returnOnEquity = np.append(returnOnEquity, financial["netIncome"]/financial["shareholderEquity"])
    return returnOnEquity

In [137]:

def netProfitMarginSerice(financials):
    #Net profit margin = (Net income/ Revenu)
    if not("financials" in financials.keys()) or len(financials["financials"]) == 0:
        return []
    financialsData = financials["financials"]
    netProfitMargin = np.array([])
    for financial in financialsData:
        if "totalRevenue" in financial.keys() and "netIncome" in financial.keys() : 
            netIncome = financial["netIncome"]
            totalRevenue = financial["totalRevenue"]
            if netIncome is not None and totalRevenue is not None:
                netProfitMargin = np.append(netProfitMargin, float(netIncome)/float(totalRevenue))
    return netProfitMargin

In [138]:
def cashFlowSerice(financials):
    if not("financials" in financials.keys()) or len(financials["financials"]) == 0:
        return []
    financialsData = financials["financials"]
    cashFlows = np.array([])
    for financial in financialsData:
        if "cashFlow" in financial.keys()  : 
            cashFlow = financial["cashFlow"]
            if cashFlow is not None:
                cashFlows = np.append(cashFlows,cashFlow )
    return cashFlows

In [171]:
def GuruFocusIntrinsicValue(ticker):
    #Intrinsic Value=(Growth Multipler * Free Cash Flow (6 year avg)+Total Stockholders Equity (Jun20)*0.8)/Shares Outstanding (Diluted Average)
    financials = callApi("financials", ticker, True)
    financial = callApi("financials", ticker)
    #financials["financials"].insert(0, financial["financials"][0])
    stats = callApi("stats", ticker)
    
    earnings = callApi("earnings", ticker, True)
    #earning = callApi("earnings", ticker)
    #earnings["earnings"].insert(0,earning["earnings"][0])

    # EPS COMPOUND ANNUAL GROWTH RATES (CAGR)
    epsCAGR = EPS_CAGR(earnings)
    print(epsCAGR)
    
    #Free Cash Flow 
    free_cash_flow = cashFlowSerice(financials)
    mean_free_cash_flow = statistics.mean(free_cash_flow)/1000000
    print(mean_free_cash_flow)
    
    # Total Stockholders Equity
    stockholders_equity = shareholdersEquity(financial)/1000000
    print(stockholders_equity)
    
    # Sharesoutstanding
    shares_outstanding = sharesoutstanding(stats)/1000000
    
    iv = (epsCAGR*100* mean_free_cash_flow + stockholders_equity *0.8)/shares_outstanding
    print(iv)

In [164]:
def BenjaminGrahamsIntrinsicValue(ticker,price, earnings):
    financials = callApi("financials", ticker, True)
    financial = callApi("financials", ticker)
    financials["financials"].insert(0, financial["financials"][0])
    stats = callApi("stats", ticker)

    
    #ROE = Net Income / Shareholder Equity
    roe = returnOnEquitySerice(financials)
    print(roe)
    if len(roe) > 0:  
        medianROE = statistics.mean(roe)
    else:
        return "NA"
    print(medianROE)
    
    #Net profit margin = (Net income/ Revenu)
    netProfitMargin = netProfitMarginSerice(financials)
    print(netProfitMargin)
    if len(netProfitMargin) > 0:
        meanNetProfit  = statistics.mean(netProfitMargin)
    else: 
        return "NA"
    
    # EPS COMPOUND ANNUAL GROWTH RATES (CAGR)
    epsCAGR = EPS_CAGR(earnings)
    print(epsCAGR)
    # Shareholders equity
    shareholders_equity = shareholdersEquity(financial)
    print(shareholders_equity)
    
    # Sharesoutstanding
    shares_outstanding = sharesoutstanding(stats)
    print(shares_outstanding)
    
    # Total Revenue
    total_revenue = totalRevenue(financial)
    print(total_revenue)
    
    # 20 AA Corporate bond Rate %
    #https://ycharts.com/indicators/us_corporate_aa_effective_yield
    Bond_Yeild_AAA = 1.51
    
    # Current Stock Price
    current_stock_price = price
    print(current_stock_price)
    
    # Required margin of safety
    margin_of_safety = 30
    
    #************************************#
    # Normalized EPS Using The ROE Method
    
    #Normalized Net Income
    net_income_roe = medianROE * shareholders_equity
    # EPS = net income / share 
    eps_roe = net_income_roe / shares_outstanding
    
    #****************************************#
    #Normalized EPS Using The Net Margin Method
    net_income_on_net_margin = meanNetProfit * total_revenue
    eps_net_income_on_net_margin = net_income_on_net_margin / shares_outstanding
    
    #****************************************#
    #Average EPS 
    average_EPS = statistics.mean([eps_roe, eps_net_income_on_net_margin] )
    #Estimated Future EPS Growth Rate
    future_EPS_growth_rate = epsCAGR
    
    
    ########################################
    #Intrinsic value = [EPS × (7 + 1 *g ) × 4.4]/Y 
    intrinsicValue = (average_EPS * (7 + 1 * future_EPS_growth_rate) *4.4) / Bond_Yeild_AAA
    if intrinsicValue <= 0:
        return "NA"
    else: 
        return intrinsicValue

In [183]:
tickers = ['HTZ']
dataSet = [] 
for ticker in tickers:
    data = {}
    points = 0
    data["Ticker"] = ticker
    # Major data set 
    quote = callApi("quote", ticker)
    stats = callApi("advanced-stats", ticker)
    balance_sheet = callApi("balance-sheet", ticker)
    earnings = callApi("earnings", ticker, True)
    earning = callApi("earnings", ticker)
    #earnings["earnings"].insert(0,earning["earnings"][0])
    price = callApi("price", ticker)
    
    data["Price"] = price
    # PE Ratio < 15
    pe_ratio = peRatio(quote)
    if pe_ratio != 0: 
        data["PE Ratio < 15"] = pe_ratio
    else: 
        data["PE Ratio < 15"] = "NA"

    # Price To Book Ratio  
    price_to_book = priceToBook(stats)
    if price_to_book != 0: 
        data["Price To Book < 1.5"] = price_to_book
    else: 
        data["Price To Book < 1.5"] = "NA"
    
    if pe_ratio * price_to_book < 22.5 and pe_ratio * price_to_book > 0: 
        points += 1
    
    # PEG Ratio 
    peg_ratio = pegRatio(stats)
    if peg_ratio != 0: 
        data["PEG Ratio(1 and 0)"] = peg_ratio
    else: 
        data["PEG Ratio(1 and 0)"] = "NA"
    if peg_ratio < 1.1 and peg_ratio > 0:
        points += 1
    
    # Debt To Equity Ratio 
    # Warent buffet say < 0.5
    debt_Equity = debtToEquity(stats)
    if debt_Equity != 0: 
        data["Debt to Equity (0 and 1.5)"] = debt_Equity
    else: 
        data["Debt to Equity (0 and 1.5)"] = "NA"
    if debt_Equity < 1.6 and debt_Equity > 0:
        points += 1
        
    # Current Ratio 
    current_ratio = currentRatio(balance_sheet)
    if current_ratio != 0: 
        data["Current Ratio > 1.5"] = current_ratio
    else: 
        data["Current Ratio > 1.5"] = "NA"
        
    if current_ratio > 1.4:
        points += 1
    
    # Earning Progression Last 4 year 
    isProgressing = checkErningProgression(earnings)
    data["Earning Progression"] =  isProgressing
    if isProgressing:
        points += 1
    
    data["Points(5)"] = points
    
    #Intrinsic value  
    data["Intrinsic value"] = BenjaminGrahamsIntrinsicValue(ticker,price,earnings)
    dataSet.append(data.copy())
pd.DataFrame(dataSet)    

[-1.51520572 -0.03278689 -0.21206409  0.21513158 -0.45674419  0.13521545]
-0.31107564235652047
[-1.01802885 -0.00593108 -0.02367424  0.03714643 -0.05577644  0.02591362]
0.353716394173877
559000000
156206000.0
832000000
1.36


,Ticker,Price,PE Ratio < 15,Price To Book < 1.5,PEG Ratio(1 and 0),Debt to Equity (0 and 1.5),Current Ratio > 1.5,Earning Progression,Points(5),Intrinsic value
0,HTZ,1.36,-0.17,0.123623,NA,10.7,1.029987,True,1,NA


In [184]:
GuruFocusIntrinsicValue('HTZ')

Unknown expeption onHTZ
Unknown expeption onHTZ
0.0


StatisticsError: mean requires at least one data point